# Contents Based Filtering 실습
## ( TMBD 5000 영화 데이터 셋 )
## ✔ 장르 속성 이용한 contents기반 필터링 예제

In [12]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [13]:
movies=pd.read_csv('C:/Users/kimjh/d/tmdb_5000_movies.csv')
movies.shape

(4803, 20)

In [14]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


## (1) Feature Selection
- id, title, genres, vote_average, vote_count, popularity , vote_average, vote _count, popularity
- keyword, overview

In [15]:
mdf=movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

### ✔ data 전처리 
- genres 같은 column은 여러개의 값을 한 칸에 가지고 있음 
- 개별 장르의 명칭을 KEY 인 name으로 추출 
- ast module의 literal_eval() 함수 이용

In [16]:
from ast import literal_eval
mdf['genres']= mdf['genres'].apply(literal_eval)
mdf['keywords']= mdf['keywords'].apply(literal_eval)

In [18]:
mdf.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",7.2,11800,150.437577,"[{'id': 1463, 'name': 'culture clash'}, {'id':...","In the 22nd century, a paraplegic Marine is di..."


In [19]:
mdf['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

- genres column이 문자열이 아닌 여러 genres dictionary 객체 가짐 
- genres에서 Action, Adventure 같은 value값만 꺼내오기

In [20]:
mdf['genres']=mdf['genres'].apply(lambda x: [y['name'] for y in x])
mdf['keywords']=mdf['keywords'].apply(lambda x: [y['name'] for y in x])
mdf[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


## (2) genres contents 유사도 측정 
### genres column처럼 여러개의 값이 list로 구성된 경우 장르별 유사도를 측정하는 방법
1. genres를 문자열로 변경
2. CountVectorizer로 feature Vectorization한 행렬 data 값을 consine 유사도로 비교
3. genres 유사도가 높은 영화 중에 평점이 높은 순으로 영화 추천

# sklearn의 CountVectorizer 이용해 feature vector 행렬로 만들기

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
mdf['genres_literal']=mdf['genres'].apply(lambda x:(' '.join(x)))

In [23]:
print(mdf['genres'][0])
print(mdf['genres_literal'][0])

['Action', 'Adventure', 'Fantasy', 'Science Fiction']
Action Adventure Fantasy Science Fiction


## Feature vectorization

In [24]:
count_vect=CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat= count_vect.fit_transform(mdf['genres_literal'])

In [31]:
genre_mat.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## 코사인 유사도 
- Feature Vectorization 된 행렬에 consine_similarities 적용

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
genre_sim=cosine_similarity(genre_mat, genre_mat)

In [34]:
genre_sim.shape

(4803, 4803)

In [35]:
genre_sim[:1]

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ]])

## genre_sim 객체의 유사도 값이 높은 순으로 정렬된 행렬의 위치 index 값 추출 
- argsort () 이용


In [36]:
genre_sim_sorted_ind=genre_sim.argsort()[:,::-1]

In [38]:
genre_sim_sorted_ind[:1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401]], dtype=int64)

In [40]:
genre_sim_sorted_ind[:2]

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401]], dtype=int64)

### Result
- 0번 레코드의 경우 자신인 0번 레코드를 제외하면 3494번 record가 가장 유사도가 높고 그 다음이 813번 레코드임
- 가장 유사도가 낮은 레코드는 2401번 레코드임

## Contents based Filtering 이용한 영화 추천
- 장르 유사도에 따라 영화 추천하는 함수 생성
- genre_sim_sorted_ind 
    - record 별 장르 코사인 유사도 index 가지는 list
- 고객이 선정한 추천 기준이 되는 영화 제목, 영화 건수 입력하면 추천 영화 정보 가지는 df 반환

In [43]:
def find_sim_movie(df,sorted_ind, title_name, top_n=10):
    # mdf DF에서 title값을 가진 row 추출
    title_movie=df[df['title']==title_name]
    # 추출한 movie의 값을 array형태로 반환
    title_idx=title_movie.index.values
    #  해당 영화 cosine 유사도가 높은 순로 top_n개 반환
    similar_indexes= sorted_ind[title_idx, :(top_n)]

    # 추출된 top_n개의 index 출력 (2차원 data)
    # DF에서 index로 사용하기 위해 1차원 arr로 변경
    print(similar_indexes)
    similar_indexes= similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [48]:
similar_movies= find_sim_movie(mdf, genre_sim_sorted_ind, "The Godfather",3)
similar_movies['title']

[[2731 1243 3636]]


2731    The Godfather: Part II
1243              Mean Streets
3636             Light Sleeper
Name: title, dtype: object